### Bi-LSTM Conditional Random Field Discussion

讲的非常清楚，但是通常的实现功能少于CRF++(发射矩阵相当于U当前特征，转移矩阵相当于B特征，而CRF++可以定义多个U模板特征，和 B**，其他特征)  
https://createmomo.github.io/2017/09/12/CRF_Layer_on_the_Top_of_BiLSTM_1/  
https://createmomo.github.io/2017/09/23/CRF_Layer_on_the_Top_of_BiLSTM_2/  
https://createmomo.github.io/2017/10/08/CRF-Layer-on-the-Top-of-BiLSTM-3/  
https://createmomo.github.io/2017/10/17/CRF-Layer-on-the-Top-of-BiLSTM-4/  
https://createmomo.github.io/2017/11/11/CRF-Layer-on-the-Top-of-BiLSTM-5/  
https://createmomo.github.io/2017/11/24/CRF-Layer-on-the-Top-of-BiLSTM-6/  
https://createmomo.github.io/2017/12/06/CRF-Layer-on-the-Top-of-BiLSTM-7/  
https://createmomo.github.io/2017/12/07/CRF-Layer-on-the-Top-of-BiLSTM-8/  
https://github.com/createmomo/CRF-Layer-on-the-Top-of-BiLSTM

For this section, we will see a full, complicated example of a Bi-LSTM Conditional Random Field for named-entity recognition.  The LSTM tagger above is typically sufficient for part-of-speech tagging, but a sequence model like the CRF is really essential for strong performance on NER.  Familiarity with CRF's is assumed.  Although this name sounds scary, all the model is is a CRF but where an LSTM provides the features.  This is an advanced model though, far more complicated than any earlier model in this tutorial.  If you want to skip it, that is fine.  To see if you're ready, see if you can:

* Write the recurrence for the viterbi variable at step i for tag k.
* Modify the above recurrence to compute the forward variables instead.
* Modify again the above recurrence to compute the forward variables in log-space (hint: log-sum-exp)

If you can do those three things, you should be able to understand the code below.
Recall that the CRF computes a conditional probability.  Let $y$ be a tag sequence and $x$ an input sequence of words.  Then we compute
$$ P(y|x) = \frac{\exp{(\text{Score}(x, y)})}{\sum_{y'} \exp{(\text{Score}(x, y')})} $$

Where the score is determined by defining some log potentials $\log \psi_i(x,y)$ such that
$$ \text{Score}(x,y) = \sum_i \log \psi_i(x,y) $$
To make the partition function tractable, the potentials must look only at local features.

In the Bi-LSTM CRF, we define two kinds of potentials: emission and transition.  The emission potential for the word at index $i$ comes from the hidden state of the Bi-LSTM at timestep $i$.  The transition scores are stored in a $|T|x|T|$ matrix $\textbf{P}$, where $T$ is the tag set.  In my implementation, $\textbf{P}_{j,k}$ is the score of transitioning to tag $j$ from tag $k$.  So:

$$ \text{Score}(x,y) = \sum_i \log \psi_\text{EMIT}(y_i \rightarrow x_i) + \log \psi_\text{TRANS}(y_{i-1} \rightarrow y_i) $$
$$ = \sum_i h_i[y_i] + \textbf{P}_{y_i, y_{i-1}} $$
where in this second expression, we think of the tags as being assigned unique non-negative indices.

If the above discussion was too brief, you can check out [this](http://www.cs.columbia.edu/%7Emcollins/crf.pdf) write up from Michael Collins on CRFs.

### The Forward Algorithm in Log-Space and the Log-Sum-Exp Trick

As hinted at above, computing the forward variables requires using a log-sum-exp.  I want to explain why, since it was a little confusing to me at first, and many resources just present the forward algorithm in potential space.  The recurrence for the forward variable at the $i$'th word for the tag $j$, $\alpha_i(j)$, is
$$ \alpha_i(j) = \sum_{j' \in T} \psi_\text{EMIT}(j \rightarrow i) \times \psi_\text{TRANS}(j' \rightarrow j) \times \alpha_{i-1}(j') $$

This is numerically unstable, and underflow is likely.  It is also inconvenient to work with proper non-negative potentials in our model.  We instead want to compute $\log \alpha_i(j)$.  What we need to do is to multiply the potentials, which corresponds to adding log potentials.  Then, we have to sum over tags, but what is the corresponding operation to summing over tags in log space?  It is not clear.  Instead, we need to transform out of log-space, take the product of potentials, do the sum over tags, and then transform back to log space.  This is broken down in the revised recurrence below:

$$ \log \alpha_i(j) = \log \overbrace{\sum_{j' \in T} \exp{(\log \psi_\text{EMIT}(j \rightarrow i) + \log \psi_\text{TRANS}(j' \rightarrow j) + \log \alpha_{i-1}(j'))}}^\text{transform out of log-space and compute forward variable} $$

If you carry out elementary exponential / logarithm identities in the stuff under the overbrace above, you will see that it computes the same thing as the first recurrence, then just takes the logarithm.  Log-sum-exp appears a fair bit in machine learning, and there is a [well-known trick](https://en.wikipedia.org/wiki/LogSumExp) to computing it in a numerically stable way.  I use this trick in my log_sum_exp function below (I don't think Pytorch provides this function in its library).

### Implementation Notes

The example below implements the forward algorithm in log space to compute the partition function, and the viterbi algorithm to decode.  Backpropagation will compute the gradients automatically for us.  We don't have to do anything by hand.

The implementation is not optimized.  If you understand what is going on, you'll probably quickly see that iterating over the next tag in the forward algorithm could probably be done in one big operation.  I wanted to code to be more readable.  If you want to make the relevant change, you could probably use this tagger for real tasks.

https://github.com/mahatmaWM/NCRFpp/blob/master/model/crf.py

In [ ]:
# -*- coding: utf-8 -*-

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)


# Helper functions to make the code more readable.
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)


def get_tags(seq, tag_to_ix):
    ix_to_tag = {v: k for k, v in list(tag_to_ix.items())}
    res = [ix_to_tag[w] for w in seq]
    return res


def to_scalar(var):
    # returns a python float
    return var.view(-1).data.tolist()[0]


def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return to_scalar(idx)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + torch.log(
        torch.sum(torch.exp(vec - max_score_broadcast)))


class BiLSTM_CRF(nn.Module):
    '''
    定义一句话的crf层，注意没有batch的概念
    '''

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        print("build single sentence crf...")
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        # 依次定义embedding、bilstm、hidden_to_tag层（这层后面接CRF）
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(
            embedding_dim,
            int(hidden_dim / 2),
            num_layers=1,
            bidirectional=True)

        # 每个token最后一维有tagset_size个，相当于是这个token到tag空间的发射得分
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # 转移矩阵，Entry i,j is the score of transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000
        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (autograd.Variable(torch.randn(2, 1, self.hidden_dim)),
                autograd.Variable(torch.randn(2, 1, self.hidden_dim)))

    def _forward_alg(self, feats):
        '''
        前向传播计算一句sentence的得分
        :param feats: (sentence_len, tagset_size)
        :return:
        '''
        init_alphas = torch.Tensor(1, self.tagset_size).fill_(-10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
        # Wrap in a variable so that we will get automatic backprop
        forward_var = autograd.Variable(init_alphas)
        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward variables at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of the previous tag
                emit_score = feat[next_tag].view(1, -1).expand(
                    1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the edge (i -> next_tag)
                # before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the scores.
                alphas_t.append(log_sum_exp(next_tag_var))
                # print 'alphas_t=', alphas_t
            forward_var = torch.cat([item.unsqueeze(0) for item in alphas_t],
                                    dim=0).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(
            len(sentence), 1, -1)  # (sentence_len, 1, embedding_dim)
        lstm_out, self.hidden = self.lstm(embeds)
        lstm_out = lstm_out.view(len(sentence),
                                 self.hidden_dim)  # (sentence_len, hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)  # (sentence_len, tagset_size)
        # print(lstm_feats)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        '''
        在当前参数下，计算给定sentence，tags的得分
        :param feats: (sentence_len, tagset_size)
        :param tags: (tagset_size)
        :return:
        '''
        # print('tags=', tags)
        score = autograd.Variable(torch.Tensor([0]))
        tags = torch.cat([torch.LongTensor([self.tag_to_ix[START_TAG]]), tags])
        for i, feat in enumerate(feats):
            score = score + self.transitions[tags[i + 1], tags[i]] + feat[tags[
                i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        '''
        每个token最后一维有tagset_size个，相当于是这个token到tag空间的发射得分
        :param feats: (sentence_len, tagset_size)
        :return:
        '''
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.Tensor(1, self.tagset_size).fill_(-10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = autograd.Variable(init_vvars)
        for feat in feats:
            bptrs_t = []  # holds the back pointers for this step
            viterbi_vars_t = []  # holds the viterbi variables for this step

            # 计算到下一个状态的转移得分，并保存最优的
            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the previous step,
                # plus the score of transitioning from tag i to next_tag.

                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbi_vars_t.append(next_tag_var[0][best_tag_id])
            # Now add in the emission scores, and assign forward_var to the set of viterbi variables we just computed
            # print('viterbi_vars_t=', viterbi_vars_t)
            # 将上面计算的最优的转移得分，加上当前token的发射得分
            forward_var = (torch.cat(
                [item.unsqueeze(0)
                 for item in viterbi_vars_t], dim=0) + feat).view(1, -1)
            # print('forward_var=', forward_var)

            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        # 计算完所有的token的forward_var之后，转移到STOP TAG，得到最终序列得分path_score
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        # 保存最优路径
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        '''

        :param feats: (sentence_len, tagset_size)
        :param tags: (tagset_size)
        :return:
        '''
        self.hidden = self.init_hidden()
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    # 损失函数有很多种定义方式
    # 下面的作业对应的改动，更换一种损失函数定义
#     def neg_log_likelihood(self, sentence, tags):
#         self.hidden = self.init_hidden()
#         feats = self._get_lstm_features(sentence)
#         score, tag_seq = self._viterbi_decode(feats)
#         gold_score = self._score_sentence(feats, tags)
#         return score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        self.hidden = self.init_hidden()
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq


# 开始训练
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

# Make up some training data
training_data = [
    ("the wall street journal reported today that apple corporation made money"
     .split(), "B I I I O O O B I O O".split()),
    ("georgia tech is a university in georgia".split(),
     "B I O O O O B".split())
]

word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
precheck_tags = torch.LongTensor([tag_to_ix[t] for t in training_data[0][1]])
print(training_data[0][0], training_data[0][1])
out = model(precheck_sent)
print('before train=', out)
print('before train=', get_tags(out[1], tag_to_ix))

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(200):
    for sentence, tags in training_data:
        model.zero_grad()

        # 形状 tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.LongTensor([tag_to_ix[t] for t in tags])
        neg_log_likelihood = model.neg_log_likelihood(sentence_in, targets)
        neg_log_likelihood.backward()
        optimizer.step()

out = model(precheck_sent)
print('after train=', out)
print('after train=', get_tags(out[1], tag_to_ix))